In [4]:
import os
import pickle
import pandas as pd

# ——— 1) Veriyi yükle ve kopyala ———
cache_dir = "weather_cache"
pkl_path   = os.path.join(cache_dir, "all_years_merged.pkl")
with open(pkl_path, "rb") as f:
    df = pickle.load(f)
df = df.copy()

# ——— 2) Çok eksik sütunları at ———
missing_ratio = df.isnull().mean()
to_drop       = missing_ratio[missing_ratio > 0.20].index.tolist()
print("Atılan sütunlar (%20’den fazla eksik):", to_drop)
df.drop(columns=to_drop, inplace=True)

# ——— 3a) Sürekli meteoroloji değişkenleri → medyan ———
cont_cols = ['t2m', 'u10', 'v10', 'd2m', 'sp', 'tp', 'Speed']
for col in cont_cols:
    med = df[col].median()
    df[col] = df[col].fillna(med)

# ——— 3b) Kategorik/grup değişkenleri → mod ———
cat_cols = ['ArrDel15', 'ArrivalDelayGroups', 'DepDel15', 'DepartureDelayGroups']
for col in cat_cols:
    mode = df[col].mode()[0]
    df[col] = df[col].fillna(mode)

# ——— 3c) Tüm saat sütunları için ileri/geri doldurma ———
time_cols = ['CRSArrTime_time', 'DepTime_time', 'ArrTime_time']
for col in time_cols:
    df[col] = df[col].ffill().bfill()

# ——— 4) Son kontrol ———
print("Kalan eksik değer var mı?:", df.isnull().any().any())
print("Her sütundaki kalan eksik değer sayısı:\n", 
      df.isnull().sum().loc[lambda x: x>0])


Atılan sütunlar (%20’den fazla eksik): ['WheelsOff_time', 'WheelsOn_time', 'TaxiIn_log', 'TaxiOut_log']
Kalan eksik değer var mı?: False
Her sütundaki kalan eksik değer sayısı:
 Series([], dtype: int64)


In [5]:
df.to_pickle("weather_cache/last_all_years_cleaned.pkl")